In [3]:
import os, sys, logging, pickle
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from typing import Dict
from functools import partial
import pyproj
import numpy as np
gpd.options.use_pygeos = False

In [6]:
ports_df = pd.read_csv('./../results_backup/output/port_node_dataframe.csv')

In [5]:
railway_df = pd.read_csv('./../results_backup/output/railway_nodes_dataframe.csv')

In [9]:
merged_df = pd.read_csv('./../results_backup/prm_ports_merged_with_railways.csv')

/home/ubuntu/miniconda3/envs/ffsc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,7,8,9,11,12,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
merged_df.network_coordinates = merged_df.network_coordinates.str[1:-1].str.split(', ')

In [19]:
railway_df

,coordinates,RailwayNodeID:ID(RailwayNode),lat,long,:LABEL
0,"(-158.15219300029378, 21.387556000051557)",railway_node_-158.1521930002937821.38755600005...,-158.152193,21.387556,RailwayNode
1,"(-158.15216799970239, 21.38780900006117)",railway_node_-158.1521679997023921.38780900006117,-158.152168,21.387809,RailwayNode
2,"(-158.15211900047154, 21.3871660001145)",railway_node_-158.1521190004715421.3871660001145,-158.152119,21.387166,RailwayNode
3,"(-158.15187400009557, 21.38819900025087)",railway_node_-158.1518740000955721.38819900025087,-158.151874,21.388199,RailwayNode
4,"(-158.15187399963196, 21.386891000336703)",railway_node_-158.1518739996319621.38689100033...,-158.151874,21.386891,RailwayNode
...,...,...,...,...,...
5219765,"(179.59374990853584, -16.321222378757763)",railway_node_179.59374990853584-16.32122237875...,179.593750,-16.321222,RailwayNode
5219766,"(179.59446716456043, -16.324194025562907)",railway_node_179.59446716456043-16.32419402556...,179.594467,-16.324194,RailwayNode
5219767,"(179.59683231035925, -16.279861429547147)",railway_node_179.59683231035925-16.27986142954...,179.596832,-16.279861,RailwayNode
5219768,"(179.60139458135137, -16.283721943758717)",railway_node_179.60139458135137-16.28372194375...,179.601395,-16.283722,RailwayNode


In [32]:
def coord_to_rail_key(coord):
    return "railway_node_" + "".join([str(item) for item in coord])

def port_item_to_node_id(id):
    return "port_node_" + str(int(id))

def create_port_railway_edges(prm_ports_matched_with_railways):
    edges = prm_ports_matched_with_railways[["item_id", "network_coordinates"]]

    edges = edges.rename({"item_id": "PortNode:START_ID(PortNode)"}, axis=1)
    edges["PortNode:START_ID(PortNode)"] = edges["PortNode:START_ID(PortNode)"].apply(
        port_item_to_node_id
    )

    edges["RailwayNodeID:END_ID(PipelineNode)"] = edges["network_coordinates"].apply(
        coord_to_rail_key
    )

    edges = edges.drop("network_coordinates", axis=1)

    edges[":TYPE"] = "PORT_RAILWAY_CONNECTOR"

    return edges

def geodesic_point_buffer(lat, lon, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return Polygon(transform(project, buf).exterior.coords[:])

In [33]:
edges_df = create_port_railway_edges(merged_df)

In [37]:
## make sure edge railways nodes are in the ID of railway node
check_isin = edges_df['RailwayNodeID:END_ID(PipelineNode)'].isin(railway_df['RailwayNodeID:ID(RailwayNode)'])

In [42]:
np.sum(~check_isin)

0

In [43]:
edges_df.to_csv('./../results_backup/output/port_railway_edge_dataframe.csv')